In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import B_ID, whichBisSig, D0_decay_type, whichBisSig_NAHS

## load the NN

In [2]:
nn_vars = ["px","py","pz","E","M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"]

In [3]:
sys.path.append('/afs/desy.de/user/a/axelheim/private/baumbauen/notebooks/')
from BranchSeparatorModel import BranchSeparatorModel
# See below why I put this



model_dir="/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/saved_models/NAHSA_Gmodes_fixedD0modes/NAHS_allEvts_twoSubs_fixedD0run/NAHSA_allExtras/256_0_64_0.1_4/"
checkpoint_name = "model_checkpoint_model_perfectSA=0.7651.pt"
specs_output_label = "256_0_64_0.1_4"
num_classes = 3    


specs = specs_output_label.split("_")

model = BranchSeparatorModel(infeatures=len(nn_vars),
            dim_feedforward=int(specs[0]),
            num_classes=num_classes,
            dropout=float(specs[3]),
            nblocks=int(specs[4]))



import torch

checkpoint = torch.load(model_dir +  checkpoint_name, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)

Using factor graph MLP encoder.


<All keys matched successfully>

## load the online data

In [4]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/appliedNNdata/6thRun/"
#nfs_path="/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/load_NN_to_basf2/productive_method/testOut/"

In [9]:
FSPs_file = uproot.open(nfs_path + "FSPs.root:variables;1")
df_FSPs = FSPs_file.arrays(library="pd")

## add labels

In [10]:
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [11]:
Hc_motherB_df = df_FSPs[df_FSPs["NN_prediction"].isna() == True].drop_duplicates(subset=("__event__"), keep='first')
Hc_motherB_df["B_tag_ID"] = Hc_motherB_df["B_ID"]
df_FSPs = pd.merge(df_FSPs,Hc_motherB_df[["__event__","__production__","B_tag_ID"]], on=["__event__","__production__"])

In [12]:
def labels(s):
    label = -1
    if int(s['B_ID']) == 0:
        label = 0 # background, cause not related to MC Particles
    else: 
        B_tagID = s['B_tag_ID']
        
        if int(s['B_ID']) == B_tagID:
            label = 1 # X
        else:
            label = 2 # Bsig
    return label
df_FSPs['label'] = df_FSPs.apply(labels, axis=1)

In [13]:
df_FSPs["correct_pred_onlineNN"] = (df_FSPs["label"] == df_FSPs["NN_prediction"]).astype(int)

In [14]:
df_FSPs.shape[0]

934093

## prepare input for NN

In [15]:
nonHc_FSPs = df_FSPs[df_FSPs["NN_prediction"].notna() == True]

In [16]:
nonHc_FSPs["NN_prediction"].describe()

count    774405.000000
mean          1.193995
std           0.741438
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           2.000000
Name: NN_prediction, dtype: float64

In [17]:
nonHc_FSPs.shape[0]

774405

In [18]:
nonHc_FSPs["offline_NN_pred"] = -1
nonHc_FSPs["offline_NN_pred_shuffled"] = -1

<ipython-input-18-83b3aaa500f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["offline_NN_pred"] = -1
<ipython-input-18-83b3aaa500f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["offline_NN_pred_shuffled"] = -1


In [19]:
evts = nonHc_FSPs["__event__"].unique()

In [20]:
evts

array([2334642, 2335906, 2336150, ..., 7940578, 7942255, 7943556],
      dtype=int32)

In [21]:
evts[2]

2336150

In [ ]:
for i in range(len(evts)):
    one_evt = nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])]
    
    num_particles = one_evt.shape[0]
    
    tmp_par_vars = []
    
    for j in range(num_particles):
        particle = one_evt.iloc[j]
        #print(particle)
        readOut_features = [particle[var] for var in nn_vars]
        tmp_par_vars.append(readOut_features)

    NN_input_features = np.array([np.array(xi) for xi in tmp_par_vars])

    # impute the nan values with -1. (check if that's logical for all values if input vars get changed)
    NN_input_features= np.nan_to_num(NN_input_features, copy=False, nan=-1.0)

    shape = NN_input_features.shape
    NN_input_features = NN_input_features.reshape(shape[0], 1, shape[1])

    #print("NN_input_features.shape:",NN_input_features.shape)
    NN_input_features = torch.Tensor(NN_input_features)
    print(NN_input_features.shape[0])

    SA_pred = model(NN_input_features)

    probs = torch.softmax(SA_pred, dim=1)  # (N, C, d1)
    winners = probs.argmax(dim=1)
    
    
    shape = NN_input_features.shape
    r=torch.randperm(shape[0])
    shuffled_input=NN_input_features[r, :, :]
    
    shuffled_SA_pred = model(shuffled_input)

    shuffled_probs = torch.softmax(shuffled_SA_pred, dim=1)  # (N, C, d1)
    shuffled_winners = shuffled_probs.argmax(dim=1)
    print("r:",r)
    for j in range(num_particles):
        particle = one_evt.iloc[j]
        particle["offline_NN_pred"] = winners[0,j].item()
        
        index_Shuffreversed = (r == j).nonzero(as_tuple=True)[0].item()
        print("j:",j,"index_Shuffreversed:",index_Shuffreversed)
        
        particle["offline_NN_pred_shuffled"] = shuffled_winners[0,index_Shuffreversed].item()
        one_evt.iloc[j] = particle
        
    nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])] = one_evt 


<ipython-input-22-86341084fdad>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred"] = winners[0,j].item()
<ipython-input-22-86341084fdad>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred_shuffled"] = shuffled_winners[0,index_Shuffreversed].item()
/afs/desy.de/user/a/axelheim/miniconda3/envs/baum/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

12
r: tensor([ 5, 11,  6,  7,  8,  1,  3,  4,  2,  0,  9, 10])
j: 0 index_Shuffreversed: 9
j: 1 index_Shuffreversed: 5
j: 2 index_Shuffreversed: 8
j: 3 index_Shuffreversed: 6
j: 4 index_Shuffreversed: 7
j: 5 index_Shuffreversed: 0
j: 6 index_Shuffreversed: 2
j: 7 index_Shuffreversed: 3
j: 8 index_Shuffreversed: 4
j: 9 index_Shuffreversed: 10
j: 10 index_Shuffreversed: 11
j: 11 index_Shuffreversed: 1


<ipython-input-22-86341084fdad>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])] = one_evt
/afs/desy.de/user/a/axelheim/miniconda3/envs/baum/lib/python3.8/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


13
r: tensor([ 6,  0, 11,  4,  2, 12,  7,  1,  5, 10,  8,  9,  3])
j: 0 index_Shuffreversed: 1
j: 1 index_Shuffreversed: 7
j: 2 index_Shuffreversed: 4
j: 3 index_Shuffreversed: 12
j: 4 index_Shuffreversed: 3
j: 5 index_Shuffreversed: 8
j: 6 index_Shuffreversed: 0
j: 7 index_Shuffreversed: 6


<ipython-input-22-86341084fdad>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred"] = winners[0,j].item()
<ipython-input-22-86341084fdad>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred_shuffled"] = shuffled_winners[0,index_Shuffreversed].item()
/afs/desy.de/user/a/axelheim/miniconda3/envs/baum/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

...

17
r: tensor([ 2, 14,  7, 10,  4,  9, 16, 13, 15,  5,  8,  0,  6,  3, 11, 12,  1])
j: 0 index_Shuffreversed: 11
j: 1 index_Shuffreversed: 16
j: 2 index_Shuffreversed: 0
j: 3 index_Shuffreversed: 13
j: 4 index_Shuffreversed: 4
j: 5 index_Shuffreversed: 9
j: 6 index_Shuffreversed: 12
j: 7 index_Shuffreversed: 2
j: 8 index_Shuffreversed: 10
j: 9 index_Shuffreversed: 5


<ipython-input-22-86341084fdad>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred"] = winners[0,j].item()
<ipython-input-22-86341084fdad>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  particle["offline_NN_pred_shuffled"] = shuffled_winners[0,index_Shuffreversed].item()
/afs/desy.de/user/a/axelheim/miniconda3/envs/baum/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

j: 10 index_Shuffreversed: 3
j: 11 index_Shuffreversed: 14
j: 12 index_Shuffreversed: 15
j: 13 index_Shuffreversed: 7
j: 14 index_Shuffreversed: 1
j: 15 index_Shuffreversed: 8
j: 16 index_Shuffreversed: 6


<ipython-input-22-86341084fdad>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])] = one_evt
/afs/desy.de/user/a/axelheim/miniconda3/envs/baum/lib/python3.8/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


14
r: tensor([ 4,  8, 11, 10,  9,  7,  0,  5,  2,  6,  3, 12, 13,  1])
j: 0 index_Shuffreversed: 6
j: 1 index_Shuffreversed: 13
j: 2 index_Shuffreversed: 8
j: 3 index_Shuffreversed: 10
j: 4 index_Shuffreversed: 0
j: 5 index_Shuffreversed: 7
j: 6 index_Shuffreversed: 9
j: 7 index_Shuffreversed: 5
j: 8 index_Shuffreversed: 1
j: 9 index_Shuffreversed: 4


In [56]:
t = torch.Tensor([1, 2, 3])
print ((t == 2).nonzero(as_tuple=True)[0].item())

1


In [57]:
r

tensor([14,  2, 17,  1, 13,  3, 15, 16, 10, 12, 11, 18,  9,  7,  4,  8,  6,  0,
         5])

In [58]:
#r = torch.Tensor([2,1,3,0])
r = torch.Tensor([ 5,  7,  9,  8,  1, 11,  2,  4,  0, 10,  6,  3])

for j in range(len(r)):
    print(r[j].item())
    print((r == j).nonzero(as_tuple=True)[0].item(), '\n')

5.0
8 

7.0
4 

9.0
6 

8.0
11 

1.0
7 

11.0
0 

2.0
10 

4.0
1 

0.0
3 

10.0
2 

6.0
9 

3.0
5 



In [59]:
for key in nonHc_FSPs.keys():
    print(key)

__experiment__
__run__
__event__
__production__
__candidate__
__ncandidates__
__weight__
genMothPDG_0
genMothPDG_1
genMothPDG_2
genMothPDG_3
genMothPDG_4
genMothPDG_5
genMothPDG_6
genMothPDG_7
genMothPDG_8
genMothPDG_9
genMotherID_0
genMotherID_1
genMotherID_2
genMotherID_3
genMotherID_4
genMotherID_5
genMotherID_6
genMotherID_7
genMotherID_8
genMotherID_9
mcMother0_uniqParID
mcMother1_uniqParID
mcMother2_uniqParID
mcMother3_uniqParID
mcMother4_uniqParID
mcMother5_uniqParID
mcMother6_uniqParID
mcMother7_uniqParID
mcMother8_uniqParID
mcMother9_uniqParID
cmpx
cmpy
cmpz
cmE
isSignal
uniqueParticleIdentifier
mcErrors
mcPDG
genMotherID
genMotherP
genMotherPDG
charge
dr
dz
clusterReg
clusterE9E21
M
PDG
genParticleID
px
py
pz
pt
p
E
kaonID
pionID
protonID
muonID
electronID
deuteronID
binaryPID__bo211__cm__sp13__bc
binaryPID__bo211__cm__sp321__bc
binaryPID__bo211__cm__sp2212__bc
binaryPID__bo321__cm__sp2212__bc
x
y
z
NN_prediction
basf2_used
basf2_Bsig
Hc_used
B_ID
B_tag_ID
label
correct_pred_

In [60]:
nonHc_FSPs[["label","offline_NN_pred","offline_NN_pred_shuffled","NN_prediction","px"]]

,label,offline_NN_pred,offline_NN_pred_shuffled,NN_prediction,px
2,2,2,2,2.0,0.361425
3,1,1,1,1.0,1.067757
4,2,2,2,2.0,-0.516327
5,0,1,0,1.0,0.020827
6,0,0,0,0.0,-0.014110
...,...,...,...,...,...
67,2,1,2,1.0,0.008499
68,1,1,1,1.0,-0.042810
70,1,2,2,2.0,-0.114170
71,2,2,2,1.0,-0.304605


In [61]:
nonHc_FSPs["off_eq_on"] = (nonHc_FSPs["offline_NN_pred"] == nonHc_FSPs["NN_prediction"]).astype(int)
nonHc_FSPs["off_eq_on_shuffled"] = (nonHc_FSPs["offline_NN_pred_shuffled"] == nonHc_FSPs["NN_prediction"]).astype(int)
nonHc_FSPs["unshuff_eq_shuff"] = (nonHc_FSPs["offline_NN_pred_shuffled"] == nonHc_FSPs["offline_NN_pred"]).astype(int)

<ipython-input-61-b197fadaab7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["off_eq_on"] = (nonHc_FSPs["offline_NN_pred"] == nonHc_FSPs["NN_prediction"]).astype(int)
<ipython-input-61-b197fadaab7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["off_eq_on_shuffled"] = (nonHc_FSPs["offline_NN_pred_shuffled"] == nonHc_FSPs["NN_prediction"]).astype(int)
<ipython-input-61-b197fadaab7f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [62]:
nonHc_FSPs["correct_pred_offlineNN"] = (nonHc_FSPs["label"] == nonHc_FSPs["offline_NN_pred"]).astype(int)
nonHc_FSPs["correct_pred_offlineNN_shuff"] = (nonHc_FSPs["label"] == nonHc_FSPs["offline_NN_pred_shuffled"]).astype(int)

<ipython-input-62-525926a4d953>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["correct_pred_offlineNN"] = (nonHc_FSPs["label"] == nonHc_FSPs["offline_NN_pred"]).astype(int)
<ipython-input-62-525926a4d953>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonHc_FSPs["correct_pred_offlineNN_shuff"] = (nonHc_FSPs["label"] == nonHc_FSPs["offline_NN_pred_shuffled"]).astype(int)


In [63]:
for var in ["off_eq_on_shuffled","off_eq_on","unshuff_eq_shuff","correct_pred_onlineNN","correct_pred_offlineNN","correct_pred_offlineNN_shuff"]:
    print(var)
    print(nonHc_FSPs[var].mean()*100, '% \n')

off_eq_on_shuffled
73.4375 % 

off_eq_on
78.125 % 

unshuff_eq_shuff
81.25 % 

correct_pred_onlineNN
51.5625 % 

correct_pred_offlineNN
45.3125 % 

correct_pred_offlineNN_shuff
48.4375 % 



In [98]:
nonHc_FSPs["off_eq_on"].describe()

count    64.000000
mean      0.828125
std       0.380254
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: off_eq_on, dtype: float64

In [99]:
nonHc_FSPs["unshuff_eq_shuff"].describe()

count    64.000000
mean      0.796875
std       0.405505
min       0.000000
25%       1.000000
50%       1.000000
75%       1.000000
max       1.000000
Name: unshuff_eq_shuff, dtype: float64

In [24]:
NN_input_features

tensor([[[ 2.2768e-01,  5.5676e-01,  2.2666e-01,  8.1050e-01,  4.9368e-01,
           1.0000e+00,  6.3749e-03, -1.1514e-02,  2.0000e+00,  9.7852e-01,
           1.5954e-12,  1.0000e+00,  1.4011e-15,  6.1770e-14,  8.7838e-23,
          -4.2892e-02,  1.7540e-02, -2.0312e-02]],

        [[ 1.6237e-01, -1.1711e-02,  2.4510e-01,  2.9423e-01,  0.0000e+00,
           0.0000e+00,  1.1869e-09,  2.2352e-10,  2.0000e+00,  9.7070e-01,
          -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          -5.0000e-02,  1.7000e-02, -1.0000e-02]],

        [[ 3.3086e-03,  1.5348e-01,  1.7568e-01,  2.3330e-01,  0.0000e+00,
           0.0000e+00,  1.1869e-09,  2.2352e-10,  2.0000e+00,  9.9316e-01,
          -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          -5.0000e-02,  1.7000e-02, -1.0000e-02]],

        [[ 1.9024e-01,  3.3351e-01,  3.3866e-01,  5.1197e-01,  0.0000e+00,
           0.0000e+00,  1.1869e-09,  2.2352e-10,  2.0000e+00,  1.0000e+00,
          -1.0000e+

In [33]:
new

tensor([[[ 8.4322e-02, -8.8876e-02,  2.5501e-02,  1.2514e-01,  0.0000e+00,
           0.0000e+00,  1.1869e-09,  2.2352e-10,  2.0000e+00,  1.0000e+00,
          -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          -5.0000e-02,  1.7000e-02, -1.0000e-02]],

        [[-3.0460e-01, -6.3019e-02,  7.3305e-02,  3.4872e-01,  1.3957e-01,
          -1.0000e+00,  7.1923e-04, -2.2001e-02,  2.0000e+00,  8.4375e-01,
           1.0000e+00,  7.7163e-31,  3.0024e-19,  1.5534e-11,  0.0000e+00,
          -5.2822e-03,  2.5532e-02, -4.2728e-02]],

        [[ 1.6237e-01, -1.1711e-02,  2.4510e-01,  2.9423e-01,  0.0000e+00,
           0.0000e+00,  1.1869e-09,  2.2352e-10,  2.0000e+00,  9.7070e-01,
          -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
          -5.0000e-02,  1.7000e-02, -1.0000e-02]],

        [[-3.1037e-02,  1.0952e-01,  7.2676e-02,  1.9421e-01,  1.3957e-01,
          -1.0000e+00,  5.6544e-02, -2.6099e-01, -1.0000e+00, -1.0000e+00,
           5.0317e-

In [34]:
r

tensor([ 7, 17,  1, 18, 12, 11,  6,  3, 14, 13,  5, 15, 10, 16,  0,  8,  9,  2,
         4])

In [31]:
NN_input_features.shape

torch.Size([19, 1, 18])

In [30]:
shape

(19, 18)

In [26]:
r

tensor([ 7, 17,  1, 18, 12, 11,  6,  3, 14, 13,  5, 15, 10, 16,  0,  8,  9,  2,
         4])

In [47]:
shape = NN_input_features.shape
r=torch.randperm(shape[0])

In [48]:
new=NN_input_features[r, :, :]

In [49]:
SA_pred = model(NN_input_features)

probs = torch.softmax(SA_pred, dim=1)  # (N, C, d1)

winners = probs.argmax(dim=1)
winners_orig = winners[:,r]
print(winners_orig)

tensor([[1, 1, 1, 2, 0, 0, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 0, 2, 1]])


In [50]:
SA_pred = model(new)

probs = torch.softmax(SA_pred, dim=1)  # (N, C, d1)

winners = probs.argmax(dim=1)
winners_new = winners
print(winners)

tensor([[1, 1, 1, 2, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1]])


In [51]:
(winners_new == winners_orig)

tensor([[ True,  True,  True,  True,  True,  True, False,  True,  True,  True,
          True,  True, False, False,  True,  True, False,  True,  True]])

In [ ]:
for i in range(len(evts)):
    one_evt = nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])]
    
    num_particles = one_evt.shape[0]
    
    tmp_par_vars = []
    
    for j in range(num_particles):
        particle = one_evt.iloc[j]
        #print(particle)
        readOut_features = [particle[var] for var in nn_vars]
        tmp_par_vars.append(readOut_features)

    NN_input_features = np.array([np.array(xi) for xi in tmp_par_vars])

    # impute the nan values with -1. (check if that's logical for all values if input vars get changed)
    NN_input_features= np.nan_to_num(NN_input_features, copy=False, nan=-1.0)

    shape = NN_input_features.shape
    NN_input_features = NN_input_features.reshape(shape[0], 1, shape[1])

    #print("NN_input_features.shape:",NN_input_features.shape)
    NN_input_features = torch.Tensor(NN_input_features)
    print(NN_input_features.shape[0])

    SA_pred = model(NN_input_features)

    probs = torch.softmax(SA_pred, dim=1)  # (N, C, d1)
    winners = probs.argmax(dim=1)
    
    
    shape = NN_input_features.shape
    r=torch.randperm(shape[0])
    shuffled_input=NN_input_features[r, :, :]
    
    shuffled_SA_pred = model(shuffled_input)

    shuffled_probs = torch.softmax(shuffled_SA_pred, dim=1)  # (N, C, d1)
    shuffled_winners = shuffled_probs.argmax(dim=1)
    print("r:",r)
    for j in range(num_particles):
        particle = one_evt.iloc[j]
        particle["offline_NN_pred"] = winners[0,j].item()
        
        index_Shuffreversed = (r == j).nonzero(as_tuple=True)[0].item()
        print("j:",j,"index_Shuffreversed:",index_Shuffreversed)
        
        particle["offline_NN_pred_shuffled"] = shuffled_winners[0,index_Shuffreversed].item()
        one_evt.iloc[j] = particle
        
    nonHc_FSPs[(nonHc_FSPs["__event__"] == evts[i])] = one_evt 


In [100]:
l1 = [1,2,4]
l2 = [1,2,5]

l1==l2

False